In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [7]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI.git",to_path=repo_path)

In [10]:
loader = GenericLoader.from_filesystem(repo_path,
                                       glob="**/*",
                                       suffixes=[".py"],
                                       parser=LanguageParser(language=Language.PYTHON,parser_threshold=500)
                                       
                                       )

documents = loader.load()

In [11]:
len(documents)

7

In [12]:
documents

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()\n\nPINECONE_API_KEY=os.environ.get(\'PINECONE_API_KEY\')\nOPENAI_API_KEY=os.environ.get(\'OPENAI_API_KEY\')\n\nos.environ["PINECONE_API_KEY"] = PINECONE_API_KEY\nos.environ["OPENAI_API_KEY"] = OPENAI_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n\nindex_name = "medicalbot"\n\n# Embed each chunk and upsert the embeddings into your Pinecone index.\ndocsearch = PineconeVectorS

In [17]:
splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,chunk_size=500,chunk_overlap=20)
docs_chunk = splitter.split_documents(documents=documents)

len(docs_chunk)

13

In [25]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY


In [26]:
embeddings = OpenAIEmbeddings(disallowed_special=())

C:\Users\fayab\AppData\Local\Temp\ipykernel_2852\805463212.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(disallowed_special=())


In [28]:
vectordb=Chroma.from_documents(docs_chunk,embedding=embeddings,persist_directory="db")
vectordb.persist()

C:\Users\fayab\AppData\Local\Temp\ipykernel_2852\900598010.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [29]:
llm = ChatOpenAI()
memory = ConversationSummaryMemory(llm=llm,memory_key='chat_history',return_messages=True)

C:\Users\fayab\AppData\Local\Temp\ipykernel_2852\805189021.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI()


In [54]:
qa=ConversationalRetrievalChain.from_llm(llm,retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"K":6}))

In [51]:
# retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"K":8})
# docs = retriever.get_relevant_documents(question)
# print(docs)


C:\Users\fayab\AppData\Local\Temp\ipykernel_2852\201295467.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents(question)


[Document(metadata={'language': 'python', 'source': 'test_repo\\src\\helper.py'}, page_content="def text_split(extracted_data):\n    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)\n    text_chunks=text_splitter.split_documents(extracted_data)\n    return text_chunks\n\n\n\n#Download the Embeddings from HuggingFace \ndef download_hugging_face_embeddings():\n    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')  #this model return 384 dimensions\n    return embeddings"), Document(metadata={'language': 'python', 'source': 'test_repo\\store_index.py'}, page_content='extracted_data=load_pdf_file(data=\'Data/\')\ntext_chunks=text_split(extracted_data)\nembeddings = download_hugging_face_embeddings()\n\n\npc = Pinecone(api_key=PINECONE_API_KEY)\n\nindex_name = "medicalbot"\n\n\npc.create_index(\n    name=index_name,\n    dimension=384, \n    metric="cosine", \n    spec=ServerlessSpec(\n        cloud="aws", \n        region="u

In [55]:
# Initialize the chat history as an empty list
chat_history = []

# Now call the QA chain with both the question and chat history
question = "what is download_hugginface_embeddings function?"
result = qa({'question': question, 'chat_history': chat_history})

print(result['answer'])


The `download_hugging_face_embeddings` function is a function that downloads pre-trained embeddings from the Hugging Face model hub. In this case, it specifically downloads the "sentence-transformers/all-MiniLM-L6-v2" model, which returns embeddings of 384 dimensions. These embeddings can be used for tasks like semantic similarity, text classification, and more within the application.
